###### STT:
###### Họ và tên: Vũ Cao Nguyên.
###### Buổi 6 - Bài Lab 6

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

In [2]:
url = 'https://raw.githubusercontent.com/datasethub/ds105/master/Lab_Model_Development.csv'

In [3]:
df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   curb-weight  201 non-null    int64  
 1   engine-size  201 non-null    int64  
 2   length       201 non-null    float64
 3   width        201 non-null    float64
 4   horsepower   201 non-null    float64
 5   city-mpg     201 non-null    int64  
 6   highway-mpg  201 non-null    int64  
 7   wheel-base   201 non-null    float64
 8   bore         201 non-null    float64
 9   price        201 non-null    float64
dtypes: float64(6), int64(4)
memory usage: 15.8 KB


In [4]:
df.head()

,curb-weight,engine-size,length,width,horsepower,city-mpg,highway-mpg,wheel-base,bore,price
0,2548,130,0.811148,0.890278,111.0,21,27,88.6,3.47,13495.0
1,2548,130,0.811148,0.890278,111.0,21,27,88.6,3.47,16500.0
2,2823,152,0.822681,0.909722,154.0,19,26,94.5,2.68,16500.0
3,2337,109,0.848630,0.919444,102.0,24,30,99.8,3.19,13950.0
4,2824,136,0.848630,0.922222,115.0,18,22,99.4,3.19,17450.0


In [5]:
## Chạy vòng lặp để lấy các tổ hơp chập 2 đến 9 của các feature trong dataframe.
list_f = df.columns[:-1]
list_feature = list()
for i in range(1,10):
        for c in itertools.combinations(list_f, i):
    #     for c in itertools.combinations_with_replacement(list_f, i):
            if i == 0 | i == 1:
            ##Không lấy giá trị có 1 feature vì nó là simple linear regression
                continue
            feature = list(c)
            list_feature.append(feature)

In [6]:
## Lấy tổ hợp từ 2 đến 9 của các feature kết hợp với nhau để tiến hành train model cho các tổ hợp
def multiple_linear(df1,number):
    arr = np.ones((502,6))
    j = 0
    for i in range(1,10):
        for c in itertools.combinations(list_f, i):
    #     for c in itertools.combinations_with_replacement(list_f, i):
            if i == 0 | i == 1:
                continue
            feature = list(c)
            x_train, x_test, y_train, y_test = train_test_split(df1[feature], df1['price'], test_size=number, random_state=0)
            lm1 = LinearRegression()
            lm1.fit(x_train, y_train)
            yhat_train = lm1.predict(x_train)
            yhat_test = lm1.predict(x_test)
            
            RMSE = np.sqrt(mean_squared_error(y_train, yhat_train))
            
            R2_train = r2_score(y_train, yhat_train)
            R2_test = r2_score(y_test, yhat_test)

            cross_val_score4 = cross_val_score(lm1, df1[feature], df1['price'], cv= 4)
            cross_val_score5 = cross_val_score(lm1, df1[feature], df1['price'], cv= 5)
            
            arr[j, ::] = [RMSE, R2_train, R2_test,cross_val_score4.mean(), cross_val_score5.mean(), number]
            j+=1
    return arr

In [7]:
## Kết quả trả về là một mảng numpy
arr_multiple_liner = multiple_linear(df, 0.3)

In [8]:
### Đổ dữ liệu vào DataFrame với data là mảng trả về của hàm multiple_linear
df_rs_muliple = pd.DataFrame(arr_multiple_liner, columns = ['RMSE','R2_train', 'R2_test', '4-fold-validation', '5-fold-validation', 'Note'])
df_rs_muliple['Model'] ="Multiple_linear"
df_rs_muliple['Features']= list_feature
df_rs_muliple

,RMSE,R2_train,R2_test,4-fold-validation,5-fold-validation,Note,Model,Features
0,3203.634006,0.801349,0.738035,0.692598,0.564379,0.3,Multiple_linear,"[curb-weight, engine-size]"
1,3549.929774,0.756082,0.592638,0.572486,0.384783,0.3,Multiple_linear,"[curb-weight, length]"
2,3498.306441,0.763125,0.591723,0.556751,0.370275,0.3,Multiple_linear,"[curb-weight, width]"
3,3419.864403,0.773628,0.701618,0.591817,0.459520,0.3,Multiple_linear,"[curb-weight, horsepower]"
4,3550.048453,0.756066,0.597126,0.573012,0.384187,0.3,Multiple_linear,"[curb-weight, city-mpg]"
...,...,...,...,...,...,...,...,...
497,3053.390514,0.819545,0.713585,0.607131,0.507021,0.3,Multiple_linear,"[curb-weight, engine-size, length, horsepower,..."
498,3042.295203,0.820854,0.721573,0.622173,0.528872,0.3,Multiple_linear,"[curb-weight, engine-size, width, horsepower, ..."
499,3221.582630,0.799117,0.672285,0.536661,0.409787,0.3,Multiple_linear,"[curb-weight, length, width, horsepower, city-..."
500,3117.736266,0.811859,0.730035,0.630413,0.553561,0.3,Multiple_linear,"[engine-size, length, width, horsepower, city-..."


In [9]:
def simple_regression(df1,number):
    list_f = df1.columns.values[:-1]
    arr = np.ones((9,6))
    i = 0
    for feature in list_f:
        x_train, x_test, y_train, y_test = train_test_split(df1[[feature]], df1['price'], test_size=number, random_state=0)
        lm1 = LinearRegression()
        lm1.fit(x_train, y_train)
        yhat_train = lm1.predict(x_train)
        yhat_test = lm1.predict(x_test)
        
        RMSE = np.sqrt(mean_squared_error(y_train, yhat_train))
        
        R2_train = r2_score(y_train, yhat_train)
        R2_test = r2_score(y_test, yhat_test)
        
        cross_val_score4 = cross_val_score(lm1, df1[[feature]], df1['price'], cv= 4)
        cross_val_score5 = cross_val_score(lm1, df1[[feature]], df1['price'], cv= 5)
        arr[i, ::] = [RMSE,R2_train, R2_test,cross_val_score4.mean(), cross_val_score5.mean(), number]
        i+=1
    return arr

In [10]:
arr_rs = simple_regression(df,0.3)

In [11]:
df_rs = pd.DataFrame(arr_rs, columns = ['RMSE','R2_train', 'R2_test', '4-fold-validation', '5-fold-validation', 'Note'])
df_rs['Model'] ="Simple_linear"
df_rs['Features']= df.columns[:-1]
df_rs

,RMSE,R2_train,R2_test,4-fold-validation,5-fold-validation,Note,Model,Features
0,3550.202785,0.756044,0.595359,0.572579,0.372340,0.3,Simple_linear,curb-weight
1,3784.535657,0.722777,0.789319,0.675548,0.563182,0.3,Simple_linear,engine-size
2,4601.851241,0.590108,0.291082,0.267107,0.138444,0.3,Simple_linear,length
3,4280.737759,0.645316,0.431023,0.449604,0.363216,0.3,Simple_linear,width
4,4564.775626,0.596686,0.695898,0.522010,0.386388,0.3,Simple_linear,horsepower
5,5338.460380,0.448385,0.457132,0.415626,0.298165,0.3,Simple_linear,city-mpg
6,5083.149495,0.499885,0.453946,0.388166,0.314959,0.3,Simple_linear,highway-mpg
7,5245.144759,0.467500,0.126825,0.028055,-0.098708,0.3,Simple_linear,wheel-base
8,6199.055806,0.256201,0.294991,0.050694,0.049608,0.3,Simple_linear,bore


In [12]:
## Kết 2 DataFrame vào một (Linear regression đơn biến và Linear Regression đa biến).
df_concat = (pd.concat([df_rs,df_rs_muliple, ]).reset_index()).drop(columns='index')
df_concat

,RMSE,R2_train,R2_test,4-fold-validation,5-fold-validation,Note,Model,Features
0,3550.202785,0.756044,0.595359,0.572579,0.372340,0.3,Simple_linear,curb-weight
1,3784.535657,0.722777,0.789319,0.675548,0.563182,0.3,Simple_linear,engine-size
2,4601.851241,0.590108,0.291082,0.267107,0.138444,0.3,Simple_linear,length
3,4280.737759,0.645316,0.431023,0.449604,0.363216,0.3,Simple_linear,width
4,4564.775626,0.596686,0.695898,0.522010,0.386388,0.3,Simple_linear,horsepower
...,...,...,...,...,...,...,...,...
506,3053.390514,0.819545,0.713585,0.607131,0.507021,0.3,Multiple_linear,"[curb-weight, engine-size, length, horsepower,..."
507,3042.295203,0.820854,0.721573,0.622173,0.528872,0.3,Multiple_linear,"[curb-weight, engine-size, width, horsepower, ..."
508,3221.582630,0.799117,0.672285,0.536661,0.409787,0.3,Multiple_linear,"[curb-weight, length, width, horsepower, city-..."
509,3117.736266,0.811859,0.730035,0.630413,0.553561,0.3,Multiple_linear,"[engine-size, length, width, horsepower, city-..."


### Polynomial Regression

##### Polynomial đơn biến bậc 1 đến 9.
##### Polynomial đa biến bậc 1 đến 9
##### nếu bậc 1 thì trở về simple và multiple regression
##### đơn biến bậc 1 có 9 models => Tổng 9 bậc có 81 models.
##### đa biến bậc 1 có 502 models => Tổng 9 bậc có 502 *  9 = 4,518
#### Tổng là 4599 Models trong Polynomial

In [13]:
## Lấy tổ hợp từ 1 đến 9 của các feature kết hợp với nhau để tiến hành train model cho các tổ hợp
## Tiến hành thử tất cả các bậc từ 1 đến 9.
def polynomial_regression(df1,number):
    arr = np.ones((4599,6))
    j = 0
    for i in range(1,10):
        for c in itertools.combinations(list_f, i):
    #     for c in itertools.combinations_with_replacement(list_f, i):
            if i == 0 :
                continue
            for degree_number in range(1,10):
                feature = list(c)
                x_train, x_test, y_train, y_test = train_test_split(df1[feature], df1['price'], test_size=number, random_state=0)
                
                ##inti Polynomial
                poly = PolynomialFeatures(degree=degree_number, include_bias = False)
                
                ## normalize
#                 SCALER = StandardScaler()
#                 SCALER.fit(x_train)
                ## fit_transform
                x_train_poly = poly.fit_transform(x_train)
                
                ## Linear regression
                lm1 = LinearRegression()
                lm1.fit(x_train_poly, y_train)
                yhat_train = lm1.predict(x_train_poly)
                
                x_test_poly = poly.fit_transform(x_test)
                yhat_test = lm1.predict(x_test_poly)

                RMSE = np.sqrt(mean_squared_error(y_train, yhat_train))
                
                R2_train = r2_score(y_train, yhat_train)
                R2_test = r2_score(y_test, yhat_test)

                cross_val_score4 = cross_val_score(lm1, df1[feature], df1['price'], cv= 4)
                cross_val_score5 = cross_val_score(lm1, df1[feature], df1['price'], cv= 5)
                arr[j, ::] = [RMSE, R2_train, R2_test,cross_val_score4.mean(), cross_val_score5.mean(), number]
                j+=1
    return arr

In [14]:
arr_poly = polynomial_regression(df, 0.3)

In [15]:
## Lấy tổ hợp từ 1 đến 9 của các feature kết hợp với nhau để tiến hành train model cho các tổ hợp
def list_feature_polynomial_regression(df1):
    list_features = list()
    for i in range(1,10):
        for c in itertools.combinations(list_f, i):
            if i == 0 :
                continue
            for degree_number in range(1,10):
                feature = list(c)
                list_features.append(feature)
    return list_features

In [16]:
## Lấy danh sách feature gán vào Dataframe
list_ft = list_feature_polynomial_regression(df)

## Tạo một list degree gán và DataFrame
degree_number = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
degree_number= np.vstack(degree_number)
degree_number = np.tile(degree_number, (511,1))
degree_number.size

4599

In [17]:
arr_poly
df_poly = pd.DataFrame(arr_poly, columns = ['RMSE', 'R2_train', 'R2_test', '4-fold-validation', '5-fold-validation', 'Note'])
df_poly['Model'] ="Polynomial Regression"
df_poly['Features']= list_ft
df_poly['Degree']= degree_number
df_poly.to_csv('polynomial.csv')


In [18]:
df_concat = (pd.concat([df_concat,df_poly, ]).reset_index()).drop(columns='index')
df_concat = df_concat[['Model','Features', 'RMSE', 'R2_train', 'R2_test', '4-fold-validation', '5-fold-validation', 'Note', 'Degree']]
# df_concat.to_csv('total_models.csv')

In [19]:
df_concat

,Model,Features,RMSE,R2_train,R2_test,4-fold-validation,5-fold-validation,Note,Degree
0,Simple_linear,curb-weight,3550.202785,0.756044,5.953593e-01,0.572579,0.372340,0.3,NaN
1,Simple_linear,engine-size,3784.535657,0.722777,7.893186e-01,0.675548,0.563182,0.3,NaN
2,Simple_linear,length,4601.851241,0.590108,2.910818e-01,0.267107,0.138444,0.3,NaN
3,Simple_linear,width,4280.737759,0.645316,4.310227e-01,0.449604,0.363216,0.3,NaN
4,Simple_linear,horsepower,4564.775626,0.596686,6.958977e-01,0.522010,0.386388,0.3,NaN
...,...,...,...,...,...,...,...,...,...
5105,Polynomial Regression,"[curb-weight, engine-size, length, width, hors...",225.317455,0.999017,-5.823096e+08,0.620791,0.516712,0.3,5.0
5106,Polynomial Regression,"[curb-weight, engine-size, length, width, hors...",224.955588,0.999021,-8.721396e+08,0.620791,0.516712,0.3,6.0
5107,Polynomial Regression,"[curb-weight, engine-size, length, width, hors...",224.636482,0.999023,-2.147886e+09,0.620791,0.516712,0.3,7.0
5108,Polynomial Regression,"[curb-weight, engine-size, length, width, hors...",224.518982,0.999024,-1.642524e+09,0.620791,0.516712,0.3,8.0
